In [1]:
import pandas as pd

In [2]:
idb = pd.read_csv("../eventepi/data/idb.csv")
idb = idb.rename(columns=lambda x: x.strip())
idb.columns

Index(['Ausgangs- bzw. Ausbruchsland', 'Krankheitsbild(er)',
       'Datenstand für Fallzahlen gesamt*', 'Fälle gesamt*',
       'Link zur Quelle 1', 'Link zur Quelle 2', 'Link zur Quelle 3',
       'Link zur Quelle 4'],
      dtype='object')

# This notebook intends to eyeball the IDB and to justify cleaning steps in ```eventepi/idb.py```

In [3]:
# Bring into long format
idb = idb.dropna(how="all").melt(
    id_vars=[
        "Ausgangs- bzw. Ausbruchsland",
        "Krankheitsbild(er)",
        "Datenstand für Fallzahlen gesamt*",
        "Fälle gesamt*",
    ],
    value_vars=[
        "Link zur Quelle 1",
        "Link zur Quelle 2",
        "Link zur Quelle 3",
        "Link zur Quelle 4",
    ],
    var_name="url_type_idb",
    value_name="url_idb",
)

## Clean up URLs
### The following steps intend to filter only those URLs that later can be matched with scraped articles to create labels for ML algorithms

In [4]:
idb = idb[idb.url_idb.str.contains("/don|promed", na=False)]
idb = idb[idb["url_idb"].astype(str).str.contains("http")]
idb = idb.loc[idb["url_idb"] != "http://www.promedmail.org/"]

Now we only have valid URLs of WHO DON and ProMED articles. Not shown here: in ```idb.py``` the function ```_normalize_promed_urls()``` is additionally applied to match the URL layout to the scraped URL layout and replace one wronlgy written URL.

In [5]:
idb.url_idb.unique()

array(['http://www.who.int/csr/don/15-march-2018-mers-oman/en/  ',
       'http://www.who.int/csr/don/09-april-2018-listeriosis-australia/en/',
       'http://www.promedmail.org/post/5752357',
       'http://www.promedmail.org/post/5732129',
       'http://www.promedmail.org/post/5746842',
       'http://www.promedmail.org/post/5760078',
       'http://www.promedmail.org/post/5761531',
       'http://www.promedmail.org/post/5786653',
       'http://www.promedmail.org/direct.php?id=20180426.5770330',
       'http://www.promedmail.org/post/5788496',
       'http://www.promedmail.org/post/5837791',
       'http://www.promedmail.org/post/5791763',
       'http://www.promedmail.org/post/5840563',
       'http://www.promedmail.org/post/5790914',
       'http://www.promedmail.org/post/5805270',
       'http://www.promedmail.org/post/5822566',
       'http://www.promedmail.org/post/5833137',
       'http://www.promedmail.org/post/5836545',
       'http://www.promedmail.org/post/5835487',
     

## Clean up counts

We can see that case counts are entered as free text with commas, periods, and spaces in between them. Also free text is entered instead of meerly numbers. The preprocessing in ```idb.py``` removes these commas, periods, and whitespaces. If more than one number is entered such as in "0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen", the script only keeps the first digits mentioned.

In [6]:
idb["Fälle gesamt*"].unique()

array([nan, '20', '65', '2119', '19', 'mind 18', '2', '1', '24', '6',
       '12', '3', '16', '10', '995', '5', '112', '29', '67', '130',
       '1219', '132', '14', '38', '27', '272', '4',
       '0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Todesfälle bei Kühem. Schafen und Pferden0 bei Menschen.\nMehr als 50 Tod

## Clean up countries

The main problem here is the use of abbreviations and mixed English and German entries. ```idb.py``` fixes these with manually written fixes and uses a cloud based translation for the German country names to English ones.

In [7]:
idb["Ausgangs- bzw. Ausbruchsland"].unique()

array(['Oman ', 'Australien', 'Bolivien', 'Yemen', 'La Reunion',
       'Schweiz', 'Israel', 'Uganda', 'Kenia', 'Frankreich', 'Liberia',
       'Nigeria', 'Costa Rica', 'Kamerun', 'Indien', 'Iran',
       'Saudi-Arabien', 'Taiwan', 'Vereinigte Arabische Emirate',
       'Brasilien', 'Pakistan', 'USA', 'UK', 'Irak',
       'Demokratische Republik Kongo', 'Algerien', 'Peru ', 'DRC',
       'Tschechien', 'Spanien', 'Angola', 'China', nan, 'Papua-Neuguinea',
       'Vereinigtes Königreich', 'Kongo', 'Französiche Guyana',
       'Kroatien', 'Afghanistan', 'VAE Dubai', 'Myanmar', 'Haiti',
       'Kuwait', 'VAE ', 'Romänien', 'Deutschland',
       'Tschechische Republik', 'Sudan', 'Großbritannien', 'Japan',
       'Südafrika', 'Venezuela ', 'Kenya', 'Neuseeland', 'Marokko',
       'Ireland', 'Namibia', 'Hong Kong', 'Libanon',
       'Dominikanische Republik', 'Argentinien', 'Usbekistan',
       'Trinidad & Tobago', 'Peru', 'Italien'], dtype=object)

## Clean up diseases

The main problem here is the mixture of symptoms and diseases. ```idb.py``` fixes these with manually setting syndroms to NAs, translating abbreviations, and use a cloud based translation to translate words to German

In [8]:
idb["Krankheitsbild(er)"].unique()

array(['MERS', 'Listeriose', 'Pest', 'Cholera', 'Leptospirose',
       'Tularämie ', 'Tick-borne relapsing fever ', 'Denguefieber',
       'Anthrax', nan, 'Hämolytisch-urämisches Syndrom', 'Lassafieber',
       'Lassafieber ', 'Malaria', 'Affenpocken',
       'Krim-Kongo Hämorrhagisches Fieber', 'Gelbfieber',
       'Japanische-Enzephalitis', 'Rifttalfieber', 'Masern', 'Kuhpocken',
       'Polio', 'Brucellose', 'Acute encephalitis syndrome',
       'Primary Amebic Meningo-encephalitis [PAM]', 'Ebolafieber', 'TBE',
       'Legionellose', 'Hämorrhagisches Fieber', 'keine', 'AFP',
       'Influenza', 'FSME ', '?',
       'Nierenversagen; v.a. HUS, Leptospirose', 'West-Nil-Fieber',
       'Poliomyelitis', 'Husten, Fieber', 'Bilharziose', 'Fieber',
       'Röteln', 'Melioidosis', 'Buruli-Ulkus', 'Hepatitis A', 'Sepsis',
       'Meningitis', 'NEC (Nekrotisierende Enterokolitis)', 'Tollwut',
       'Mumps', ' ', 'Hepatitis E', 'Ebola', 'Guillain-Barré Syndrom',
       'Lungenentzündung', 'Nor